# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import pandas as pd
import numpy as np
from azureml.core import Workspace, Dataset, Experiment, ScriptRunConfig
from azureml.core.environment import Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.environment import CondaDependencies
import joblib
import os
import shutil
import logging
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model, InferenceConfig
import requests
import json

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required


subscription_id = '610d6e37-4747-4a20-80eb-3aad70a55f43'
resource_group = 'aml-quickstarts-165256'
workspace_name = 'quick-starts-ws-165256'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='ethdata')
dataset.to_pandas_dataframe()

In [2]:


ws = Workspace(subscription_id, resource_group, workspace_name)

experiment_name = 'hyptuneth'

exp=Experiment(ws, experiment_name)

env=Environment(name='myenv')

conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_pip_package("pillow==5.4.1")
conda_dep.add_conda_package("pandas")
env.python.conda_dependencies=conda_dep

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-165256
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-165256


In [3]:
cluster_name = "AIcompinst3"



try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute target...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-12-07T13:41:40.713000+00:00', 'errors': None, 'creationTime': '2021-12-07T13:41:40.192881+00:00', 'modifiedTime': '2021-12-07T13:41:45.873625+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_amount = 0.2,
                      evaluation_interval=1,
                      delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        # There's only one parameter, so grid sampling will try each value - with multiple parameters it would try every combination
        '--epsilon': uniform( 1e-8,1.0),
        '--max_iter': choice(range(50,1000,25))
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory='.',
              script='train.py',
              arguments=['--input_data',dataset.as_named_input('ethdata')],              
              compute_target = compute_target,
              environment=env)
              

hyperdrive_run_config = HyperDriveConfig(
                                      hyperparameter_sampling=param_sampling, 
                                      policy=early_termination_policy,
                                      run_config=estimator,  
                                      primary_metric_name='RMSE', 
                                      primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                                      max_total_runs=25,
                                      max_concurrent_runs=4)

In [6]:
#TODO: Submit your experiment
hyprun = exp.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hyprun).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
#TODO: Save the best model

model = hyprun.get_best_run_by_primary_metric()
model = model.register_model(model_name='hyptun_eth', model_path='outputs/hyptun_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [29]:
inference_config = InferenceConfig(entry_script='score.py',environment=env)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config, overwrite=True, show_output=True)
service.wait_for_deployment(show_output = True)
print(service.state)

Models: ['hyptun_eth:1']
Entry script: score.py
Environment variables: {'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE', 'AZUREML_ENTRY_SCRIPT': 'score.py'}
Environment dependencies: ['python=3.6.2', ordereddict([('pip', ['azureml-defaults', 'pillow==5.4.1'])]), 'scikit-learn', 'pandas']
Environment docker image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210806.v1
CPU requirement: 1, Memory requirement: 1GB
Uploading dependency score.py.
Request submitted, please run wait_for_deployment(show_output=True) to get deployment status.
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-07 16:14:53+00:00 Creating Container Registry if not exists.
2021-12-07 16:14:53+00:00 Registering the environment.
2021-12-07 16:14:54+00:00 Use the existing image.
2021-12-07 16:14:54+00:00 Generating deployment configuration.
2021-12-07 16:14:55+00:00 Submitting depl

TODO: In the cell below, send a request to the web service you deployed to test it.

In [30]:


# URL for the web service
scoring_uri = '{}'.format(service.scoring_uri)
# If the service is authenticated, set the key or token
key = '<your key or token>'

# Two sets of data to score, so we get two results back
data = {"data":
        [
            [
                3587.0
            ],
            [
                3511.0
                ]
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

[3587.0613945536525, 3511.073064814104]


TODO: In the cell below, print the logs of the web service and delete the service

In [31]:
print(service.scoring_uri)
print(service.swagger_uri)
service.delete()

http://0704250d-e8a5-4fad-8dad-ef3fbe8f342d.southcentralus.azurecontainer.io/score
http://0704250d-e8a5-4fad-8dad-ef3fbe8f342d.southcentralus.azurecontainer.io/swagger.json
